# PICRUSt predictions for the GCMP data

Goal is to generate some KO table predictions for the GCMP data.




In [ ]:
#Set up the inputs
#I copied over the contents of the 1_canonical_starting_files folder on Aug 9 2016 into input.
#I also put the rep_set97 from greengenes in there for reference 
#cp /macqiime/anaconda/lib/python2.7/site-packages/qiime_default_reference/gg_13_8_otus/rep_set/97_otus.fasta ./input/
#NOTE: this comes with macqiime, but I figured having it there might make the analysis easier to duplciate on non-macqiime 
#installations


In [2]:
#Clear output folder

#CAUTION: Quoting the below rm command out so no one deletes
#output by accident when replicating

#!rm -rf ../output/* 

In [13]:
#Import relevant libraries
from os.path import join,abspath

#First normalize by copy number
input_otu_table = "../input/otu_table_mc2_wtax_no_pynast_failures_no_organelles_even1000.biom"
output_folder = "../output"
bdiv_prefs_file = "../input/bdiv_bc_prefs.txt"
mapping_file = "../input/gcmp16S_map_r23.txt"

## Filter out denovo OTUs for PICRUSt
The strategy is to filter our 'open reference' OTU table down to the equivalent of a 'closed-reference' table,
using the fasta of 97% Greenegenes OTUs as our reference 

In [2]:
filtered_otu_table = join(output_folder,'otu_table_no_organelles_even_1000_gg13_5_only.biom')
greengenes_fasta = '../input/97_otus.fasta'

!filter_otus_from_otu_table.py -i $input_otu_table -o $filtered_otu_table -e '../input/97_otus.fasta' --negate_ids_to_exclude


In [3]:
print filtered_otu_table
!ls ../output

../output/otu_table_no_organelles_even_1000_gg13_5_only.biom
otu_table_no_organelles_even_1000_gg13_5_only.biom


In [5]:
#Normalize OTU counts by predicted copy number

normalized_otu_table_filename = "normalized_otu_table_no_organelles_even1000.biom"
normalized_otu_table = join(output_folder,normalized_otu_table_filename)
normalize_cmd = "normalize_by_copy_number.py -i %s -o %s" %(filtered_otu_table,normalized_otu_table)
print normalize_cmd
!$normalize_cmd

normalize_by_copy_number.py -i ../output/otu_table_no_organelles_even_1000_gg13_5_only.biom -o ../output/normalized_otu_table_no_organelles_even1000.biom


In [10]:
output_types = ['ko','cog']
output_type_category_md = {'ko':'KEGG_Pathways','cog':'COG_Category'}
output_type_category_levels = {'ko':[1,2,3],'cog':[1]}

for output_type in output_types:
    #Make a prediction folder in output for each prediction type
    prediction_folder = join(output_folder,"picrust_%s" %output_type)
    !mkdir $prediction_folder
    prediction_file = join(prediction_folder,"%s_predictions.biom" %output_type)
    nsti_file = join(prediction_folder,"%s_nstis.txt" %output_type)                     
    !predict_metagenomes.py -i $normalized_otu_table -o $prediction_file -a $nsti_file -t $output_type
    #Decide whether to use KEGG_Pathways or COG_Category
    output_type_category = output_type_category_md[output_type]
    for level in output_type_category_levels[output_type]:
        categorized_file = join(prediction_folder,"%s_categories_L%i.biom" %(output_type,level))
        categorized_file_tsv = join(prediction_folder,"%s_categories_L%i.tsv" %(output_type,level))
        !categorize_by_function.py -i $prediction_file -l $level -o $categorized_file --metadata_category $output_type_category
        !biom convert --to-tsv -i $categorized_file -o $categorized_file_tsv
        

mkdir: ../output/picrust_ko: File exists
mkdir: ../output/picrust_cog: File exists


In [ ]:
#Checked output as follows:
#biom convert -i ko_predictions.biom -o ko_predictions.txt --to-tsv --tsv-metadata-formatter naive --header-key KEGG_Pathways


In [11]:
#Beta-diversity ordinations by each predicted function type

output_types = ['ko','cog']
for output_type in output_types:
    prediction_folder = join(output_folder,"picrust_%s" %output_type)
    prediction_file = join(prediction_folder,"%s_predictions.biom" %output_type)
    bdiv_folder = prediction_file + "_bray_curtis_results"
    !beta_diversity_through_plots.py  -i $prediction_file -o $bdiv_folder -p $bdiv_prefs_file -m $mapping_file -f
    #Seeing a lot of horseshoe effects mapping to an unclear metadata category in bdiv results
    curr_pc_file = join(bdiv_folder,'bray_curtis_pc.txt')
    #Needed currently because of QIIME issue in detrend.py  
    #see https://github.com/biocore/qiime/issues/2035
    
    reformatted_pc_file = join(bdiv_folder,'bray_curtis_pc_formatted.txt')
    !python ./convert_pc_format.py $curr_pc_file $reformatted_pc_file
    
    #Now we detrend in an unsupervised manner
    new_pc_folder = join(bdiv_folder,'bray_curtis_pc_detrended')
    !detrend.py -i $reformatted_pc_file -o $new_pc_folder 
    detrended_pcoa = join(new_pc_folder,"detrended_pcoa.txt")
    
    #I really wanted to make a detrended emperor plot, but it looks like the detrended pcoas a) have only
    #PC 1 and 2, and b) don't have the proper headers about eigenvalues etc to work in emperor.
    #I think I can still at least check a few explicit variables to see if they explain the trend
    
    #Can also try NMDS.  Why not?
    distance_matrix = join(bdiv_folder,"bray_curtis_dm.txt")
    nmds_output = join(bdiv_folder,"bray_curtis_nmds")
    !nmds.py -i $distance_matrix -o $nmds_output
    
    
    
    
   

/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.211440951834 and the largest is 17.2591663079.
  RuntimeWarning
/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see

##Machine learning on functional groups

In [15]:

for predicted_cat in ['functional_group_sensu_darling','host_genus_id','BiologicalMatter']:
    for output_type in output_types:
        prediction_folder = join(output_folder,"picrust_%s" %output_type)
        prediction_file = join(prediction_folder,"%s_predictions.biom" %output_type)
        random_forest_functional_group_dir = join(prediction_folder,"random_forest_%s_%s"%(predicted_cat,output_type))
        #NOTE: temporarily commenting this out while rerunning
        !supervised_learning.py -i $prediction_file -o $random_forest_functional_group_dir -c $predicted_cat -m $mapping_file -f
        if output_type == 'ko':
            levels = [1,2,3]
        elif output_type == 'cog':
            levels = [1]
        for level in levels:
            prediction_file = join(prediction_folder,"%s_categories_L%i.biom"%(output_type,level))
            random_forest_functional_group_dir = join(prediction_folder,"random_forest_%s_%s_%i"%(predicted_cat,output_type,level))
            !supervised_learning.py -i $prediction_file -o $random_forest_functional_group_dir -c $predicted_cat -m $mapping_file -f
            